In [4]:
import os
from pathlib import Path
dataset_folder = Path("./../../data/heart-disease") #cwd is the notebook folder

if not dataset_folder.exists():
    os.mkdir(dataset_folder)
else:
    print("Folder already exists.")

Folder already exists.


In [5]:
import pandas as pd

In [6]:
dataset_path = dataset_folder / "heart.csv"

In [7]:
df = pd.DataFrame()

if dataset_path.exists():
    df = pd.read_csv(dataset_path)

In [8]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
